# REU-CFS: Day 1, Assertions and Loop Invariants

_Burton Rosenberg, University of Miami_

_Monday, 17 May 2021_

------

This is a first program in Python. Examples are given then an exercise.

------


### Assertions and Loop Invariants


In this section we will discuss matters that pertain to all programming languges. We will discuss a certain method for getting correct code. We will exercise those methods to help write two classic sorting algorithems: selection sort and insertion sort.

An _assertion_ is a logical statement that must be true if the code is correctly functionning.

A block of code as _preconditions_, which are the assertions that it requires, and _postconditions_, where are the assertions it guarantess.

The method conceptualizes programming as the sequencing of code blocks so postconditions of the directly preceeding block match the preconditions of the directly following block.

Python has an assert statement, that can run the assertion, which is otherwise just something a programmer keeps abstractly in mind. In is a good way to test correctness, and a good way to localize a bug when something goes wrong. 

(In production code the assertions must be removed, because when an assertion fails the computer goes geek-speak, and your retail audience will not be impressed.)

Here follows an example. The distance function is the absolute value of the difference of two numbers. For the purposes of this example, we avoid the absolute value by placing the numbers in a known value order and doing a signed subtraction, which we know will come out postitive.

Step one, then, is placing the numbers in the known value ordering. Step two is calculating a weakened form of the distance function, one which assumes the first numberis never larger than the second. The step one sets up a postcondition that is the precondition needed for correctness for step two.



In [4]:


def distance(x,y):
    """
    find the distance between x and y
    """
    
    # two blocks of code, the postcondition of the first matches the
    # precondition of the second.
    # the __purpose__ of the first is to make all preconditions of the
    # second block true. 
    # we will check this with python's assert statement

    # this also demonstrates pythons multiple assignments
    x, y = min(x,y), max(x,y)
    
    # postcondition x<=y

    assert x<y
    
    # precondition for corrects (since we do not use absolute value): x<=y
    return y-x


def test_distance():
    if distance(7,3)==distance(3,7):
        print("at least it is symmetric")
    else:
        print("broken!")

test_distance()

at least it is symmetric


### Loop Invariants

Loops (for, while, self-recursion) can be difficult to prove correct. They are even difficult to prove that the loop will not get stuck and run forever, that there is an eventual exiting of the loop, i.e. that the loop terminates. So we break the correctness proof into two elements,

* that the loop would be correct if loop terminates, and
* that the loop terminates

Thinking about loops in this way already improves coding. Non-terminating loops are common, and they might show up after the program is considered correct, because a sly combination of values and events occurs, which send the termination reasoning off its unjustifiably optimistic course.

Mentally unwind the loop by placing it as a self-reproduced sequence of blocks,

>  while block-x  (is transformed into)  block-x block-x ....

Which we know will be a finite sequence but that is not our interest now. By the language of pre- and postconditions the loop block's precondition equals its postcondition, and this summarizes entirely correctness. The common assertion is called the _loop invariant_.

If the loop invariant holds, and termination is given, then the loop is correct. Here it is in pseudo-mathematics,

> LoopInvariant + Termination => Goal

The Loop Invariant is defined to be true before the loop block is ever entered. Usually this is done by making an Loop Invariant that is _trivially true_ for the loop is ever entered. The each time through the loop, the code makes progress towards the goal but perhaps disturbing the invariant while it works, but it returns it true at the bottom of the loop, as the prerequisite for the next entry to the loop.

The Loop Invariant must be written to summarize the point of the loop, so that at termination one can conclude that the goal has been reached. Often the Loop Invariant is too complicated to actually write an Python assert statement, and is written in prose or combined prose-mathematics.

Here is an example that is a bit contrived, but it will allow us the demonstrate the loop invariant with actuall assert statements.


In [48]:

def find_min(a):
    """
    find the minimum in list a using loop invariants
    """
    
    # min only exists for non-empty lists, that is a precondition
    if len(a)==0 : return None
    # postcondition, the list is non-empty
    
    # check the precondition
    assert len(a)>0 
    
    cur = a[0]  # this is correct because a is non-empty
    
    # L.I. cur is the minimum over the first i elements of the list a
    
    # L.I. trivially true for the first 1 elements
    assert cur==min(a[:1])

    for i in range(2,len(a)):
        # L.I. might not be true here! i just got one bigger!
        if a[i-1]<cur:
            # if so, fix it
            cur = a[i-1]

        # assert L.I.
        assert cur==min(a[:i])

    # Loop terminates because the loop condition is finite
    # termination implies cur == min(a[:n]) which is equal to min(a)
    return cur

def test_find_min():
    test = [19*i%33 for i in range(1,33)]
    ans = min(test)
    if find_min(test)==ans:
        print("correct!")
    else:
        print("broken!")
    
test_find_min()

correct!


### Bonus discussion: data structure invariants

The method of loop invariants can be extended to data. A _datastructure invariant_ is an assertion about the values in the datastructure that can be made before and after an update to the datastructure.

:) that's all about this for now, we want to do some exercises.


### Exercise: Classic n-squared Sorts


__Selection Sort__

Please write a selection sort, which works inplace &mdash; the numbers are rearranged in the same array, not copied to a new array. 

The helpful loop invariant is that the loop begins with i indexing the next location to be finalized, beginning with i = 0; and the value to place in location i is found among locations j>=i. If the value to put in location i comes from location j, swap values between locations i and j.


In [3]:

# fix my broken code

def selection_sort(a):
    """
    selection sort the list a
    """
    for i in range(len(a)+1):
        for j in range(i+1,len(a)):
            if a[i]>a[j]:
                temp = a[i]
                a[i]=a[j]
                a[j]=temp
    return a

def test_selection_sort():
    test = [(13*i)%97 for i in range(84)]
    ans = sorted(test[:])
    selection_sort(test)
    if test == ans:
        print("correct!")
    else:
        print("broken!")   

test_selection_sort()

correct!


__Insertion Sort__

Please write an insertion sort. The sort by its nature works in-place &mdash; the values are moved around the array, not copied off to a new array.

The Loop Invariant is that at the top of the loop, location i is considered for swapping with location i+1, to bring the smaller value to location i. Additionally j is set initially to i and the inner loop invariant is that the swapping continues towards smaller j until the value at location j is less than that of location j+1.

In [16]:
#fix my broken code

def insertion_sort(c):
    if len(c)== 0 : return c
    
    for i in range(len(c)):
        switch=False
        for j in range(i-1,-1,-1):
            if c[j]<=c[i]:
                if switch==True:
                    temp = c[i]
                    c.pop(i)
                    c.insert(j+1,temp)
                break
            elif c[j]>c[i]:
                switch=True
    return c

def test_insertion_sort():
    test = [(13*i)%97 for i in range(84)]
    ans = sorted(test[:])
    insertion_sort(test)
    if test == ans:
        print("correct!")
    else:
        print("broken!")   

test_insertion_sort()

correct!


### Exercise: Classic n-log(n) Sorts


__Merge Sort__

Please write a merge sort.


In [1]:
#fix my broken code

def merge_them(c1,ch):
    
    cf = []
    counter = 0
    count=len(c1)+len(ch)
    
    while(counter<count):
        if len(c1)==0:
            cf=cf+ch
            break
        elif len(ch)==0:
            cf=cf+c1
            break
        elif c1[0]<ch[0]:
            cf.append(c1[0])
            c1.pop(0)
        else:
            cf.append(ch[0])
            ch.pop(0)
        counter+=1

    return cf


def merge_sort(c):
    if len(c)<2 : 
        return c

    m =len(c)//2
    c= merge_them(merge_sort(c[:m]),merge_sort(c[m:]))
    return c

def test_merge_sort():
    test = [(13*i)%97 for i in range(84)]
    ans = sorted(test[:])
    if merge_sort(test) == ans:
        print("correct!")
    else:
        print("broken!")   

test_merge_sort()


[0] [13]
[0, 13]
[39] [52]
[39, 52]
[26] [39, 52]
[26, 39, 52]
[0, 13] [26, 39, 52]
[0, 13, 26, 39, 52]
[65] [78]
[65, 78]
[7] [20]
[7, 20]
[91] [7, 20]
[7, 20, 91]
[65, 78] [7, 20, 91]
[7, 20, 65, 78, 91]
[0, 13, 26, 39, 52] [7, 20, 65, 78, 91]
[0, 7, 13, 20, 26, 39, 52, 65, 78, 91]
[33] [46]
[33, 46]
[72] [85]
[72, 85]
[59] [72, 85]
[59, 72, 85]
[33, 46] [59, 72, 85]
[33, 46, 59, 72, 85]
[14] [27]
[14, 27]
[1] [14, 27]
[1, 14, 27]
[53] [66]
[53, 66]
[40] [53, 66]
[40, 53, 66]
[1, 14, 27] [40, 53, 66]
[1, 14, 27, 40, 53, 66]
[33, 46, 59, 72, 85] [1, 14, 27, 40, 53, 66]
[1, 14, 27, 33, 40, 46, 53, 59, 66, 72, 85]
[0, 7, 13, 20, 26, 39, 52, 65, 78, 91] [1, 14, 27, 33, 40, 46, 53, 59, 66, 72, 85]
[0, 1, 7, 13, 14, 20, 26, 27, 33, 39, 40, 46, 52, 53, 59, 65, 66, 72, 78, 85, 91]
[79] [92]
[79, 92]
[21] [34]
[21, 34]
[8] [21, 34]
[8, 21, 34]
[79, 92] [8, 21, 34]
[8, 21, 34, 79, 92]
[47] [60]
[47, 60]
[86] [2]
[2, 86]
[73] [2, 86]
[2, 73, 86]
[47, 60] [2, 73, 86]
[2, 47, 60, 73, 86]
[8, 21, 